In [2]:
import json
import pandas as pd
import os
import numpy as np

with open('../01_scraping/json/espn_scores.json') as rosters:
    data = json.load(rosters)

df = pd.json_normalize(data)
df

,season,week,awayteam,hometeam,awayscore,homescore,gamecast,boxscore,idgame,awayteam global record,awayteam away record,hometeam global record,hometeam home record
0,2018,1,Falcons,Eagles,12,18,https://espn.com/nfl/game/_/gameId/401030710,https://espn.com/nfl/boxscore/_/gameId/401030710,401030710,0-1,0-1,1-0,1-0
1,2018,1,Steelers,Browns,21,21,https://espn.com/nfl/game/_/gameId/401030718,https://espn.com/nfl/boxscore/_/gameId/401030718,401030718,0-0-1,0-0-1,0-0-1,0-0-1
2,2018,1,Bengals,Colts,34,23,https://espn.com/nfl/game/_/gameId/401030717,https://espn.com/nfl/boxscore/_/gameId/401030717,401030717,1-0,1-0,0-1,0-1
3,2018,1,Titans,Dolphins,20,27,https://espn.com/nfl/game/_/gameId/401030716,https://espn.com/nfl/boxscore/_/gameId/401030716,401030716,0-1,0-1,1-0,1-0
4,2020,1,Texans,Chiefs,20,34,https://espn.com/nfl/game/_/gameId/401220225,https://espn.com/nfl/boxscore/_/gameId/401220225,401220225,0-1,0-1,1-0,1-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,2021,18,49ers,Rams,27,24,https://espn.com/nfl/game/_/gameId/401326599,https://espn.com/nfl/boxscore/_/gameId/401326599,401326599,10-7,6-3,12-5,5-3
1276,2021,18,Patriots,Dolphins,24,33,https://espn.com/nfl/game/_/gameId/401326592,https://espn.com/nfl/boxscore/_/gameId/401326592,401326592,10-7,6-2,9-8,6-3
1277,2021,18,Seahawks,Cardinals,38,30,https://espn.com/nfl/game/_/gameId/401326597,https://espn.com/nfl/boxscore/_/gameId/401326597,401326597,7-10,4-5,11-6,3-5
1278,2021,18,Panthers,Buccaneers,17,41,https://espn.com/nfl/game/_/gameId/401326596,https://espn.com/nfl/boxscore/_/gameId/401326596,401326596,5-12,3-6,13-4,7-1


In [3]:
df.drop(['boxscore', 'gamecast',"awayteam global record","awayteam away record","hometeam global record","hometeam home record"], axis=1, inplace=True)
df.reset_index()
df

,season,week,awayteam,hometeam,awayscore,homescore,idgame
0,2018,1,Falcons,Eagles,12,18,401030710
1,2018,1,Steelers,Browns,21,21,401030718
2,2018,1,Bengals,Colts,34,23,401030717
3,2018,1,Titans,Dolphins,20,27,401030716
4,2020,1,Texans,Chiefs,20,34,401220225
...,...,...,...,...,...,...,...
1275,2021,18,49ers,Rams,27,24,401326599
1276,2021,18,Patriots,Dolphins,24,33,401326592
1277,2021,18,Seahawks,Cardinals,38,30,401326597
1278,2021,18,Panthers,Buccaneers,17,41,401326596


In [67]:
#df = df.loc[((df.awayteam=='Texans') | (df.hometeam=='Texans')) & (df.season=='2020')]
#from calendar import week


from operator import indexOf


df.week = df.week.astype(int)
df.awayscore = df.awayscore.astype(int)
df.homescore = df.homescore.astype(int)
df.season = df.season.astype(int)
df = df.sort_values(by=['season', 'week'],ascending = [True, True])
#   - Addition of analysis features:

#df.insert(df.shape[1],"score_abs",df["homescore"] - df["awayscore"]) #Absolute score of the game
df = df.assign(scoreabs=df.awayscore-df.homescore)
df = df.assign(winner=df.hometeam)
df= df.assign(loser=df.awayteam)
df.winner = df[['awayteam','hometeam','scoreabs']].apply(lambda x: x['awayteam'] if x['scoreabs']>0 else x['hometeam'], axis=1)
df.loser = df[['awayteam','hometeam','scoreabs']].apply(lambda x: x['awayteam'] if x['scoreabs']<0 else x['hometeam'], axis=1)

#df = df.assign(streak_home=0)
#df = df.assign(streak_away=0)
#df.streak_home = df[['awayteam','hometeam','winner']].apply(lambda x: 1 if x['hometeam']==x['winner'] else 0, axis=1)
#df.streak_away = df[['awayteam','hometeam','winner']].apply(lambda x: 1 if x['awayteam']==x['winner'] else 0, axis=1)

df_teams_stats_away = pd.DataFrame().assign(season=df.season, week=df.week, idgame=df.idgame, awayteam=df.awayteam,hometeam=df.hometeam, team=df.awayteam, winner=df.winner, loser=df.loser)
df_teams_stats_home = pd.DataFrame().assign(season=df.season, week=df.week, idgame=df.idgame, awayteam=df.awayteam,hometeam=df.hometeam, team=df.hometeam, winner=df.winner, loser=df.loser)

#df_teams_stats.append({'season' : df.season, 'week' : df.week, 'idgame' : df.idgame, 'team' : df.hometeam, 'winner' : df.winner},ignore_index=True)

#def f(x):
    #x['streak2'] = (x['winner'] == x['team']).cumsum()
    #x['cumsum'] = np.nan
    #x.loc[x['winner'] != x['team'], 'cumsum'] = df['streak2']
    #x['cumsum'] = x['cumsum'].fillna(method='ffill')
    #x['cumsum'] = x['cumsum'].fillna(0)
    #x['streak_after'] = x['streak2'] - x['cumsum']
    #x.drop(['streak2', 'cumsum'], axis=1, inplace=True)
    #return x

def f(x):
    #x['streak'] = x.groupby( (x['loser']!=x['team']).cumsum()).cumcount()+( (x['team'] != x['loser']).cumsum() == 0).astype(int) 
    x['c'] = (x['loser'] == x['team']).cumsum()
    x['a'] = (x['c'] == 0).astype(int)
    x['b'] = x.groupby('c').cumcount()
    x['win_streak'] = x.groupby('c').cumcount()+x['a']
    x.drop(['a', 'b', 'c'], axis=1, inplace=True)
    return x

def g(x):
#x['streak'] = x.groupby( (x['loser']!=x['team']).cumsum()).cumcount()+( (x['team'] != x['loser']).cumsum() == 0).astype(int) 
    x['c'] = (x['winner'] == x['team']).cumsum()
    x['a'] = (x['c'] == 0).astype(int)
    x['b'] = x.groupby('c').cumcount()
    x['lose_streak'] = x.groupby('c').cumcount()+x['a']
    x.drop(['a', 'b', 'c'], axis=1, inplace=True)
    return x

def h(x):
#x['streak'] = x.groupby( (x['loser']!=x['team']).cumsum()).cumcount()+( (x['team'] != x['loser']).cumsum() == 0).astype(int) 
    #x['c'] = (x['streak'] == x['team']).cumsum()
    #x['a'] = (x['c'] == 0).astype(int)
    #x['b'] = x.groupby('c').cumcount()
    #x['lose_streak'] = x.groupby('c').cumcount()+x['a']
    #x.drop(['a', 'b', 'c'], axis=1, inplace=True)
    return x

#d.concat([new_df,entry])
df_teams_stats = pd.concat([df_teams_stats_away,df_teams_stats_home], ignore_index=True, sort=False)
df_teams_stats = df_teams_stats.sort_values(by=['season','team','week'],ascending=[True,True,True])
df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=False)
#df_teams_stats = df_teams_stats.assign(streak_before=0)
df_teams_stats = df_teams_stats.assign(win_streak=0)
df_teams_stats = df_teams_stats.assign(lose_streak=0)
df_teams_stats = df_teams_stats.assign(streak=0)
#df_teams_stats.streak_before = df_teams_stats[['team','winner','week']].apply(lambda x: 0 if (x['week']==1) else -1, axis=1)
#df_teams_stats.streak_after = df_teams_stats[['team','winner','week']].apply(lambda x: 1 if (x['week']==1 & (x['winner']==x['team'])) else -1, axis=1)
df_teams_stats = df_teams_stats.groupby('team', sort=False).apply(f)
df_teams_stats = df_teams_stats.groupby('team', sort=False).apply(g)
#df_teams_stats = df_teams_stats.loc[df_teams_stats.team=='Jets']
#df_teams_stats = df_teams_stats.loc[df_teams_stats.season==2017]
df_teams_stats.win_streak = df_teams_stats.win_streak.astype(str).astype(int)
df_teams_stats.lose_streak = df_teams_stats.lose_streak.astype(str).astype(int)
#df_teams_stats.streak = df_teams_stats[['win_streak','lose_streak','streak']].apply(lambda x: 0-['lose_streak'] if x['lose_streak']>0 else x['win_streak'], axis=1)
df_teams_stats.streak = df_teams_stats.win_streak - df_teams_stats.lose_streak
df_teams_stats = df_teams_stats.sort_values(by=['season','week','idgame'],ascending=[False,False,False])
df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=False)
df_teams_stats.drop(['win_streak','lose_streak'], axis=1, inplace=True)
df.reset_index()

#df_teams_stats = df_teams_stats.loc[(df_teams_stats.season==2021) & ((df_teams_stats.week==18) | (df_teams_stats.week==17)) ]
#pd.set_option('display.max_rows', None)
#df_teams_stats = df_teams_stats.sort_values(by=['season','team'],ascending=[False,False])
df_teams_stats.week = df_teams_stats.week.astype(str).astype(int)

#df_teams_stats.streak_before = df[['week','team','streak']].apply(lambda x: x['streak']+1 if (x['scoreabs']>0 else x['hometeam'], axis=1)
#df.streak_before = df[['awayteam','hometeam','scoreabs','week','streak']].apply(lambda x: x['streak']+1 if (x['scoreabs']>0 else x['hometeam'], axis=1)
#final_df = pd.DataFrame()
df_teams_stats = df_teams_stats.sort_values(by=['season','team'],ascending=[False,False])
df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=False)
df_teams_stats = df_teams_stats.assign(streak_before=0)

#for i in range(len(df_teams_stats)-1,1):
for i in df_teams_stats.index:
    if df_teams_stats["week"][i] != 1:
        df_teams_stats["streak_before"][i]=df_teams_stats["streak"][i+1]
    else:
        df_teams_stats["streak_before"][i]=0

df_teams_stats
#df_teams_stats = df_teams_stats.loc[(df_teams_stats.team=='Chargers') & (df_teams_stats.season==2021)]
#df_teams_stats.streak_before = df_teams_stats.streak_before.astype(str).astype(int)
#df_teams_stats.week = df_teams_stats.week.astype(int)
#df_teams_stats.streak_before = df_teams_stats['streak'].loc[(df_teams_stats.team=='Chargers') & (df_teams_stats.season==2021) & (df_teams_stats.week==(df_teams_stats.week)-1)]

#df_teams_stats.loc[(df_teams_stats.season==2021) & (df_teams_stats.team=='Chargers') & (df_teams_stats.week==)]

#df_teams_stats = df_teams_stats.sort_values(by=['season','week','idgame'],ascending=[True,True,True])
#df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=True)
#df_teams_stats
#df_teams_stats = df.assign(streak=0)
#df_teams_stats.streak = df_teams_stats[['team','winner','week']].apply(lambda x: 1 if (x['team']==x['winner']) else 0, axis=1)
#df_team_stats = df_teams_stats.groupby(['idgame'])
#series = df.iloc[:,7]
#df['streak'] = df.result.ne(df['winner'].shift())
#df.streak_before = df[['week','team','streak']].apply(lambda x: x['streak'].loc['week'-1] if (x['team']= else x['hometeam'], axis=1)
#df.insert(df.shape[1],"winner_home", 0) #Flagging whether the home team won (1) or lose (0)

#df.loc[:,['streak']] = df.loc[(df.winner & df.week-1)]

#def f(x):
    #x.past_week = x.winner
    #x.streak = x.groupby( (x['stat'] != 0).cumsum()).cumcount()+( (x['stat'] != 0).cumsum() == 0).astype(int) 

#df_teams_stats_home

#df.loc[df.season==2017]
#df.loc[(df.player_name=='Kyler Murray') & (df.game_id==401326597)]

#df_final = pd.DataFrame().assign(season=df_teams_stats.season, week=df_teams_stats.week, idgame=df_teams_stats.idgame, awayteam=df_teams_stats.awayteam, hometeam=df_teams_stats.hometeam, winner=df_teams_stats.winner, loser=df_teams_stats.loser, streak_away = df_teams_stats['streak_before'].loc[(df_teams_stats.awayteam==df_teams_stats.team)] , streak_home = df_teams_stats['streak_before'].loc[(df_teams_stats.hometeam==df_teams_stats.team)])
#df_final
#df_teams_stats_home = pd.DataFrame().assign(season=df.season, week=df.week, idgame=df.idgame, awayteam=df.awayteam,hometeam=df.hometeam, team=df.hometeam, winner=df.winner, loser=df.loser)

C:\Users\frede\AppData\Local\Temp\ipykernel_35892\2775591185.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams_stats["streak_before"][i]=df_teams_stats["streak"][i+1]
C:\Users\frede\AppData\Local\Temp\ipykernel_35892\2775591185.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams_stats["streak_before"][i]=0


,season,week,idgame,awayteam,hometeam,winner,loser,streak_away,streak_home
0,2021,18,401326594,Washington,Giants,Washington,Giants,-4.0,NaN
1,2021,17,401326578,Eagles,Washington,Eagles,Washington,NaN,-3.0
2,2021,16,401326567,Washington,Cowboys,Cowboys,Washington,-2.0,NaN
3,2021,15,401326542,Washington,Eagles,Eagles,Washington,-1.0,NaN
4,2021,14,401326531,Cowboys,Washington,Cowboys,Washington,NaN,4.0
5,2021,13,401326518,Washington,Raiders,Washington,Raiders,3.0,NaN
6,2021,12,401326508,Seahawks,Washington,Washington,Seahawks,NaN,2.0
7,2021,11,401326481,Washington,Panthers,Washington,Panthers,1.0,NaN
8,2021,10,401326472,Buccaneers,Washington,Washington,Buccaneers,NaN,-4.0
9,2021,8,401326447,Washington,Broncos,Broncos,Washington,-3.0,NaN
